In [49]:
from collections import defaultdict
from dataclasses import astuple, dataclass
from functools import cache
from heapq import heappop, heappush
import re

sample = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II""".splitlines()

def parse(lines):
    flows = {}
    neighbors = {}
    for line in lines:
        x = re.findall("Valve (.+) has flow rate=(.+); .*valves? (.*)", line)
        [(node, flow, dests)] = x
        neighbors[node] = dests_split = dests.split(', ')
        flows[node] = int(flow)
    return flows, neighbors

def part1(input):
    flows, neighbors = parse(input)

    @cache
    def inner(curr:str, already_opened:frozenset[str], time_remaining:int):
        if time_remaining <= 0: return 0

        value = flows[curr] * (time_remaining - 1) # start turning valve at minute 1, flow starts at minute 0
        now_opened = already_opened | set([curr]) # this line is carrying a lot of weight.  set('AA') = set('A') while set(['AA']) = set('AA')

        best = 0
        for n in neighbors[curr]:
            # try opening if unopened and there's value in it
            value_in_opening = 0
            if value > 0 and (curr not in already_opened):
                value_in_opening = value + inner(n, now_opened, time_remaining-2)
            # try just moving to a neighbor
            value_in_moving = inner(n, already_opened, time_remaining-1)
            best = max(best, value_in_opening, value_in_moving)
        return best
    return inner("AA", frozenset(), 30)

print(part1(sample))



1651


In [50]:
from aoc import read_lines

print(part1(read_lines("data/day16.txt")))

2119
